In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import tensorflow as tf
sys.path.append("../")

from pgn import PGN
from loss import coverage_loss
from batcher import batcher
from utils.config import CKPT_DIR
from utils.saveLoader import Vocab
from utils.params import get_default_params
from utils.config_gpu import config_gpu

E:\software\Anaconda3\envs\tf2.0\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [3]:
config_gpu()
params = get_default_params()
model = PGN(params)
vocab = Vocab(params["vocab_path"], params["vocab_size"])

1 Physical GPUs, 1 Logical GPUs


配置保存点管理器

In [4]:
checkpoint = tf.train.Checkpoint(Seq2Seq=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, CKPT_DIR, max_to_keep=5)
checkpoint.restore(checkpoint_manager.latest_checkpoint)
if checkpoint_manager.latest_checkpoint:
    print("Restored from {}".format(checkpoint_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

Restored from E:\GitHub\QA-abstract-and-reasoning\data\checkpoints\training_checkpoints\test_model-1


In [11]:
# from utils.train_helper import train_model
# train_model(model, vocab, params, checkpoint_manager)

In [5]:
%run ../utils/train.py

1 Physical GPUs, 1 Logical GPUs
Building vocab ...
Building the model ...
Creating the checkpoint manager
Restored from E:\GitHub\QA-abstract-and-reasoning\data\checkpoints\training_checkpoints\test_model-1
Starting the training ...
Epoch 1 Batch 1 Loss 235.3908
Epoch 1 Batch 2 Loss 217.1442
Epoch 1 Batch 3 Loss 297.5134
Epoch 1 Batch 4 Loss 310.4384
Epoch 1 Batch 5 Loss 354.3774
Epoch 1 Batch 6 Loss 235.2203
Epoch 1 Batch 7 Loss 271.4305
Epoch 1 Batch 8 Loss 343.9371
Epoch 1 Batch 9 Loss 235.1501
Epoch 1 Batch 10 Loss 299.7896
Epoch 1 Batch 11 Loss 307.5605
my break


In [7]:
params['max_train_steps']

20736

In [9]:
checkpoint_prefix = os.path.join(CKPT_DIR, "test_model")
checkpoint.save(file_prefix=checkpoint_prefix)

'E:\\GitHub\\QA-abstract-and-reasoning\\data\\checkpoints\\training_checkpoints\\test_model-1'

## 一探train_model究竟

- 摒弃`@tf.funtion`

In [4]:
epochs = params['epochs']
optimizer = tf.keras.optimizers.Adagrad(params['learning_rate'],
                                            initial_accumulator_value=params['adagrad_init_acc'],
                                            clipnorm=params['max_grad_norm'])
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
# optimizer.get_config(), loss_object.get_config()

In [5]:
def loss_function(real, pred, padding_mask):
    loss = 0
    for t in range(real.shape[1]):
        # if padding_mask:
        try:
            loss_ = loss_object(real[:, t], pred[:, t, :])
            mask = tf.cast(padding_mask[:, t], dtype=loss_.dtype)
            loss_ *= mask
            loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
            loss += loss_
        # else:
        except:
            loss_ = loss_object(real[:, t], pred[:, t, :])
            loss_ = tf.reduce_mean(loss_, axis=0)  # batch-wise
            loss += loss_
    return tf.reduce_mean(loss)

In [35]:
# @tf.funtion
def train_step(enc_inp, extended_enc_input, max_oov_len,
               dec_input, dec_target, cov_loss_wt,
               enc_pad_mask, padding_mask=None):
    batch_loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = model.call_encoder(enc_inp)
        # 第一个隐藏层输入
        dec_hidden = enc_hidden
        # 逐个预测序列
        predictions, _, attentions, coverages = model(dec_input,
                                                      dec_hidden,
                                                      enc_output,
                                                      dec_target,
                                                      extended_enc_input,
                                                      max_oov_len,
                                                      enc_pad_mask=enc_pad_mask,
                                                      use_coverage=True,
                                                      prev_coverage=None)
    
        batch_loss = loss_function(dec_target, predictions, padding_mask) + \
                     cov_loss_wt * coverage_loss(attentions, coverages, padding_mask)
        
#     variables = model.encoder.trainable_variables + model.decoder.trainable_variables + \
#                 model.attention.trainable_variables + model.pointer.trainable_variables
    variables = model.trainable_variables
    gradients = tape.gradient(batch_loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [36]:
start = time.time()
dataset = batcher(vocab, params)
total_loss = 0
step = 0

In [37]:
# 进入for 循环
# for encoder_batch_data, decoder_batch_data in dataset:
encoder_batch_data, decoder_batch_data = next(iter(dataset))

In [38]:
batch_loss = train_step(enc_inp = encoder_batch_data["enc_input"],
                        extended_enc_input = encoder_batch_data["extended_enc_input"],
                        max_oov_len = encoder_batch_data["max_oov_len"],
                        dec_input = decoder_batch_data["dec_input"],
                        dec_target = decoder_batch_data["dec_target"],
                        cov_loss_wt=0.5,
                        enc_pad_mask=encoder_batch_data["sample_encoder_pad_mask"],
                        padding_mask=decoder_batch_data["sample_decoder_pad_mask"])

进入train_step